In [2]:
SEED = 123321
MISSING = -9999

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier

In [4]:
df_train = pd.read_csv("../data/interim/train.csv")
df_test = pd.read_csv("../data/interim/test.csv")

/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# explicitly categorical features
categorical = ['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161', 'cntry']

# implicitly categorical features
categorical2 = ['v70', 'v71', 'v72', 'v73', 'v102', 'v103','v158', 'v159', 'v160', 
                'v163', 'v164', 'v169', 'v170', 'v190', 'v191','v216', 'v231']

# yes-no features
yes_no_cols = ['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
               'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
               'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
               'v248', 'v254', 'v256', 'v257']

# special features
education_encoding1 = ['v56', 'v58', 'v60', 'v62']
education_encoding2 = ['v57', 'v59', 'v61', 'v63']

occupation_encoding = ['v150', 'v151']

In [6]:
df_train["test"] = 0
df_test["test"] = 1

In [7]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)
df = df.set_index('id')

In [10]:
# create happiness ratio feature
df['v98'] = pd.to_numeric(df['v98'], 'coerce') # hapiness
df['happiness_ratio_cntry'] = df['v98'] / df.groupby('cntry')["v98"].transform(np.mean)

In [12]:
# binarize yes-no features
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["2", 2], value=0)
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["1", 1], value=1)

In [13]:
# special missing values in education encoding
for col in education_encoding1:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({55:np.nan})

for col in education_encoding2:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({5555:np.nan})

In [14]:
# take first 2 digit of occupation encoding
for col in occupation_encoding:
    df[col] = df[col].apply(lambda x: x[:2])

In [15]:
one_hot = categorical + categorical2 + occupation_encoding
numeric = [col for col in df.columns if col not in one_hot + ['id', 'satisfied', 'test']]

In [17]:
# apply one-hot encoding
for col in one_hot:
    df[col] = pd.Categorical(df[col])
df_encoded = pd.get_dummies(df, columns=one_hot, drop_first=True)

In [18]:
# normalize and impute missing
df_encoded = df_encoded.apply(pd.to_numeric, args=('coerce',))
df_encoded = df_encoded.drop(columns=df_encoded.nunique()[df_encoded.nunique() <= 1].index)
df_encoded -= df_encoded.min()
df_encoded /= df_encoded.max()
df_encoded = df_encoded.fillna(MISSING)

In [25]:
# feature selection using Extremely Randomized Tree
et = ExtraTreesClassifier(n_estimators=800, random_state=SEED)
X = df_encoded[df_encoded["test"] == 0].drop(columns=["satisfied","test"])
y = df_encoded[df_encoded["test"] == 0]["satisfied"]
et.fit(X, y)

In [29]:
feature_importance = pd.DataFrame(np.array([X.columns, et.feature_importances_]).T, 
                                  columns=['feature', 'importance']) \
                                .sort_values("importance", ascending=False) \
                                .reset_index(drop=True)

In [30]:
print(feature_importance.to_string())

                    feature   importance
0                       v98     0.021273
1     happiness_ratio_cntry    0.0190149
2                       v74   0.00818891
3                       v99   0.00814689
4                      v253   0.00810304
5                       v79   0.00707922
6                      v101    0.0066732
7                      v180   0.00610377
8                      v224    0.0060668
9                       v82   0.00595954
10                     v241   0.00549501
11                     v243   0.00539707
12                   v190_2   0.00524881
13                     v178   0.00517293
14                      v80   0.00510938
15                     v227   0.00509622
16                      v81    0.0049553
17                     v234   0.00489766
18                     v131   0.00488069
19                     v129   0.00486355
20                     v225   0.00485588
21                     v134   0.00483137
22                     v179   0.00481393
23              

In [31]:
# keep only top 350 important features
non_important_cols = feature_importance[350:]['feature']
df_encoded = df_encoded.drop(columns=non_important_cols)
df_encoded.shape

In [34]:
# save data to csv
df_encoded.to_csv("../data/processed/data_encoded_stacking_final.csv", index=True)